In [2]:
import findspark
findspark.init()

import pyspark
findspark.find()

'C:\\spark\\spark-3.2.0-bin-hadoop3.2'

In [3]:
# Create SparkSession from builder
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
                    .appName('My App') \
                    .getOrCreate()
print(spark)

In [4]:
df = spark.createDataFrame(
    [("Hemanth", 250000), ("Hem", 350000), ("Hemanth Kumar", 210000)],["Name","Salary"])
df.show()

+-------------+------+
|         Name|Salary|
+-------------+------+
|      Hemanth|250000|
|          Hem|350000|
|Hemanth Kumar|210000|
+-------------+------+



In [5]:
df.show(truncate=False)

+-------------+------+
|Name         |Salary|
+-------------+------+
|Hemanth      |250000|
|Hem          |350000|
|Hemanth Kumar|210000|
+-------------+------+



In [6]:

from pyspark.sql.functions import explode
df.select(df.name,explode(df.subjects)).show(truncate=False)


AttributeError: 'DataFrame' object has no attribute 'name'